In [1]:
import pandas as pd
import numpy as np

import os
from __future__ import print_function
import codecs
import pickle
import json

import numpy as np
import pandas as pd
import random

from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.externals import joblib
from sklearn.manifold import MDS

from scipy.cluster.hierarchy import ward, dendrogram

import nltk
from nltk.stem.snowball import SnowballStemmer
import re
import mpld3

import matplotlib.pyplot as plt

In [2]:
pwd

'/Users/steventa/Desktop/TripAdvisor/OpinRankDatasetWithJudgments/hotels'

In [5]:
rating_headers = ['doc_id','hotel_name','hotel_url','street','city','state','country','zip','class','price','num_reviews','CLEANLINESS','ROOM','SERVICE','LOCATION','VALUE', 'COMFORT','overall_ratingsource']

In [6]:
df_hotels_las_vegas = pd.read_csv('data/las-vegas.csv', names = rating_headers).drop(0)

In [7]:
df_hotels_las_vegas['overall_ratingsource'] > 4

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
df_hotels_las_vegas.sort_values(['overall_ratingsource', 'num_reviews'], axis=0, ascending=False).head()

In [106]:
df_hotels_las_vegas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 1 to 230
Data columns (total 18 columns):
doc_id                  230 non-null object
hotel_name              230 non-null object
hotel_url               230 non-null object
street                  230 non-null object
city                    230 non-null object
state                   230 non-null object
country                 230 non-null object
zip                     230 non-null object
class                   230 non-null object
price                   230 non-null object
num_reviews             230 non-null int64
CLEANLINESS             230 non-null object
ROOM                    230 non-null object
SERVICE                 230 non-null object
LOCATION                230 non-null object
VALUE                   230 non-null object
COMFORT                 230 non-null object
overall_ratingsource    230 non-null float64
dtypes: float64(1), int64(1), object(16)
memory usage: 44.1+ KB


In [7]:
# convert overall_ratingsource to float
df_hotels_san_francisco['overall_ratingsource'] = df_hotels_san_francisco['overall_ratingsource'].astype(float)

In [8]:
df_hotels_san_francisco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227 entries, 1 to 227
Data columns (total 18 columns):
doc_id                  227 non-null object
hotel_name              227 non-null object
hotel_url               227 non-null object
street                  227 non-null object
city                    227 non-null object
state                   227 non-null object
country                 227 non-null object
zip                     227 non-null object
class                   227 non-null object
price                   227 non-null object
num_reviews             227 non-null object
CLEANLINESS             227 non-null object
ROOM                    227 non-null object
SERVICE                 227 non-null object
LOCATION                227 non-null object
VALUE                   227 non-null object
COMFORT                 227 non-null object
overall_ratingsource    227 non-null float64
dtypes: float64(1), object(17)
memory usage: 33.7+ KB


In [9]:
review_headers = ['date', 'review_title', 'full_review', 'nan']

In [35]:
review = pd.read_csv('data/las-vegas/usa_nevada_las-vegas_skylofts_at_mgm_grand', 
                     names=review_headers ,sep='\t', encoding='latin-1').drop('nan', axis=1)

In [37]:
review.head()

,date,review_title,full_review
0,NaN,wow,"This hotel is totally awesome. Limo, champagne..."
1,Nov 15 2009,good value,Reasonably priced (for Las Vegas) and great lo...
2,Nov 7 2009,Possibly My Favourite Hotel Experience Ever,"Having stayed in the Burj Al Arab, Dubai and t..."
3,Oct 16 2009,Didn't want to leave,Let me preface this review by saying we had be...
4,Oct 13 2009,Skylofts 1bdrm and Bellagio 1 Bdrm Penthouse,I did two nights at skylofts and two at Bellag...


In [39]:
lst_files = os.listdir('data/las-vegas/')

In [41]:
lst_files.index('usa_nevada_las-vegas_skylofts_at_mgm_grand')

70

In [42]:
for comment in review.iloc[:3,:]['full_review']:
    print(comment)
    print()

This hotel is totally awesome. Limo, champagne, red wine and franch cheese, personnal concierge, hammam and the BED!... so much, you wanna live there!

Reasonably priced (for Las Vegas) and great location. Right by the monorail stop, so its easy to get around to some of the other hotels. We liked it better than another we stayed at in Las Vegas. Staff was very helpful and we were able to buy tickets to a show right at the hotel. We picked a show that was right in the hotel (Ka). The show was great and it was relaxing not to have to go out anywhere to get to the show.

Having stayed in the Burj Al Arab, Dubai and the Conrad, Tokyo I still think this is my favourite hotel in the world. Unobtrusive but attentive staff, amazing decor and a real feeling of been treated as a VIP from the second you arrive. We stayed for a week and everythign was perfect. I recommend this hotel to anyone, provided your pockets are deep. Go for an afternoon to Wet Republic and eat at Joel Robuchon.



In [61]:
df_hotels_las_vegas['overall_ratingsource'] = df_hotels_las_vegas['overall_ratingsource'].astype(float)

In [62]:
df_hotels_las_vegas['num_reviews'] = df_hotels_las_vegas['num_reviews'].astype(int)

In [63]:
not_neg_one = df_hotels_las_vegas['overall_ratingsource'] != -1

In [64]:
# find lowest rating
without_neg_one_df = df_hotels_las_vegas[not_neg_one]

In [65]:
# more than 3 reviews
more_than_three = without_neg_one_df['num_reviews'] > 3
without_neg_one_df = without_neg_one_df[more_than_three]

In [66]:
without_neg_one_df['overall_ratingsource'].min()

1.4666666666666668

In [67]:
# now get min
rating_min = without_neg_one_df['overall_ratingsource'] == without_neg_one_df['overall_ratingsource'].min()

In [68]:
df_min_rating = without_neg_one_df[rating_min]

In [69]:
df_min_rating.head()

,doc_id,hotel_name,hotel_url,street,city,state,country,zip,class,price,num_reviews,CLEANLINESS,ROOM,SERVICE,LOCATION,VALUE,COMFORT,overall_ratingsource
222,usa_nevada_las-vegas_motel_6_las_vegas_boulder...,motel 6 las vegas boulder highway,http://www.tripadvisor.com/ShowUserReviews-g45...,-1,las vegas,nv,usa,89121,-1,-1,6,1.0,1.0,1.6666666666666667,2.6666666666666665,1.0,0.0,1.466667


In [70]:
# look at usa_nevada_las-vegas_motel_6_las_vegas_boulder

In [75]:
review_boulder = pd.read_csv('data/las-vegas/usa_nevada_las-vegas_motel_6_las_vegas_boulder_highway', 
                     names=review_headers ,sep='\t', encoding='latin-1').drop('nan', axis=1)

In [76]:
review_boulder.head()

,date,review_title,full_review
0,Nov 5 2008,Drug needles under the bed!!!!!!!!,This motel is absolutely terrible. There was a...
1,Mar 26 2007,This place stinks - literally!!!,"We requested a non-smoking room. What a joke, ..."
2,Feb 7 2007,Rather Sleep In A Cardboard Box,My wife and I were going to stay here for the ...
3,Aug 20 2006,Yuck! Double Yuck!,If you like feeling like you're sleeping in an...
4,Jun 1 2006,The WORST place you may ever stay.,"The Motel 6 on Boulder Hwy. in Las vegas, NV i..."


In [78]:
# try to find experience of this review

one_review = '''
We stayed here for 8 nights on our trip to San Francisco from Australia, and we absolutely loved it. 
From our arrival and check-in, to check-out, service was faultless and facilities were wonderful. 
We got a 2 bedroom apartment, with 2 bathrooms, and a fully equipped kitchen. 
All appliances were state of the art, and our room also had a Bosch washer and a separate dryer, 
which was really helpful. The apartment itself was very spacious, with large windows, automatic blinds, 
3 large high-def LCD screens, cable TV with 100's of channels, DVD players - as well as a gas heater in the 
living area! We loved it so much we wished that we lived at Fairmong Heritage Place Ghirardelli Square. 
Also downstairs, there are 3 or 4 Ghirardelli chocolate cafe's and shops, which were fantastic. 
There are several restaurants as well as specialty shops at the Square, and it's only a short walk from the 
Fisherman's Wharf itself. Staff were fantastic, with the concierge doing everything he could to accommodate 
to our requests, from in-room dining, to arranging a in-residence masseuse for my wife. The Maserati town car 
was absolutely fantastic as well, as it took us to Union Square, shopping areas, or other restaurants that are 
a just a bit too far to walk to. There's a guest lounge area as well, with a high quality pool table and 
daily breakfast in the morning. We loved it, and so will you.
'''

In [81]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [82]:
# load nltk's SnowballStemmer as variabled 'stemmer'
stemmer = SnowballStemmer("english")

In [87]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [91]:
hotel_names, hotel_reviews = 'usa_nevada_las-vegas_skylofts_at_mgm_grand', one_review

In [97]:
hotel_vocab_stemmed = []
hotel_vocab_tokenized = []

allwords_stemmed = tokenize_and_stem(one_review)
hotel_vocab_stemmed.extend(allwords_stemmed)
allwords_tokenized = tokenize_only(one_review)
hotel_vocab_tokenized.extend(allwords_tokenized)

In [93]:
len(hotel_vocab_stemmed), len(hotel_vocab_tokenized)

(243, 243)

In [103]:
hotel_vocab_frame = pd.DataFrame({'words': hotel_vocab_tokenized}, index = hotel_vocab_stemmed)

In [104]:
hotel_vocab_frame

,words
we,we
stay,stayed
here,here
for,for
night,nights
on,on
our,our
trip,trip
to,to
san,san
